# Segmenting Annapolis, Maryland Neighborhoods

## First, we must import all the required libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


## This is to send the GET request

In [37]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Annapolis,_Maryland").text

## The next few line are scraping the neighborhood data from Wikipedia into a dataframe.

In [38]:
soup = BeautifulSoup(data, 'html.parser')

In [39]:
neighborhoodList = []

In [41]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [44]:
# create a new DataFrame from the list
annapolis = pd.DataFrame({"Neighborhood": neighborhoodList})

annapolis

,Neighborhood
0,"Annapolis Neck, Maryland"
1,"Arnold, Maryland"
2,"Cape St. Claire, Maryland"
3,"Edgewater, Maryland"
4,"Hillsmere Shores, Maryland"
5,"Parole, Maryland"
6,"St. Margaret's, Maryland"


In [45]:
annapolis.shape

(7, 1)

## The next lines are for getting the geographical coordinates using Geocoder.

In [46]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Annapolis, Maryland'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [47]:
coords = [ get_latlng(neighborhood) for neighborhood in annapolis["Neighborhood"].tolist() ]

In [48]:
coords

[[38.980693741764, -76.48892293533038],
 [38.980693741764, -76.48892293533038],
 [38.980693741764, -76.48892293533038],
 [38.94355780699901, -76.56726959361801],
 [38.93979000000007, -76.49535999999995],
 [38.997037000000034, -76.51121399999994],
 [39.018738892228, -76.46660870177809]]

In [49]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [51]:
# merge the coordinates into the original dataframe
annapolis['Latitude'] = df_coords['Latitude']
annapolis['Longitude'] = df_coords['Longitude']

In [52]:
print(annapolis.shape)
annapolis

(7, 3)


,Neighborhood,Latitude,Longitude
0,"Annapolis Neck, Maryland",38.980694,-76.488923
1,"Arnold, Maryland",38.980694,-76.488923
2,"Cape St. Claire, Maryland",38.980694,-76.488923
3,"Edgewater, Maryland",38.943558,-76.567270
4,"Hillsmere Shores, Maryland",38.939790,-76.495360
5,"Parole, Maryland",38.997037,-76.511214
6,"St. Margaret's, Maryland",39.018739,-76.466609


## Utilized the Geopy library to get the latitude and longitude values of Annapolis, Maryland

In [55]:
# get the coordinates of Annapolis
address = 'Annapolis, Maryland'

geolocator = Nominatim(user_agent="annapolis")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Annapolis, Maryland is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Annapolis, Maryland is 38.9786401, -76.492786.


## I then created a map of Annapolis and the surrounding neighborhoods that are marked on the map utilizing folium.

In [58]:
# create map of Annapolis using latitude and longitude values
map_annapolis = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(annapolis['Latitude'], annapolis['Longitude'], annapolis['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_annapolis)  
    
map_annapolis

## I then connected to my Foursquare account

In [1]:
CLIENT_ID = '~' # your Foursquare ID
CLIENT_SECRET = '~' # your Foursquare Secret
ACCESS_TOKEN = '~' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ~
CLIENT_SECRET:~


## This is to create a function to get venues for all the neighborhoods in Annapolis, Maryland.

In [60]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## This is to run the function on each neighborhood and to create a new dataframe.

In [61]:
names= annapolis.loc[:,'Neighborhood']
longitudes= annapolis['Longitude']
latitudes= annapolis['Latitude']
annapolis_venues= getNearbyVenues(names, latitudes, longitudes)
annapolis_venues.head()

Annapolis Neck, Maryland
Arnold, Maryland
Cape St. Claire, Maryland
Edgewater, Maryland
Hillsmere Shores, Maryland
Parole, Maryland
St. Margaret's, Maryland


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Annapolis Neck, Maryland",38.980694,-76.488923,William Paca House & Garden,38.979551,-76.488065,History Museum
1,"Annapolis Neck, Maryland",38.980694,-76.488923,Galway Bay Irish Restaurant,38.979817,-76.489844,Irish Pub
2,"Annapolis Neck, Maryland",38.980694,-76.488923,Old Fox Books & Coffeehouse,38.980587,-76.488890,Bookstore
3,"Annapolis Neck, Maryland",38.980694,-76.488923,Iron Rooster,38.978300,-76.487243,Comfort Food Restaurant
4,"Annapolis Neck, Maryland",38.980694,-76.488923,Paws Pet Boutique,38.979274,-76.489909,Pet Store


In [62]:
pd.set_option('display.max_rows', None)
annapolis_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Annapolis Neck, Maryland",30,30,30,30,30,30
"Arnold, Maryland",30,30,30,30,30,30
"Cape St. Claire, Maryland",30,30,30,30,30,30
"Edgewater, Maryland",15,15,15,15,15,15
"Hillsmere Shores, Maryland",2,2,2,2,2,2
"Parole, Maryland",4,4,4,4,4,4
"St. Margaret's, Maryland",1,1,1,1,1,1


## This is how many unique categories that were returned from the venues

In [63]:
print('There are {} uniques categories.'.format(len(annapolis_venues['Venue Category'].unique())))

There are 45 uniques categories.


## This is setting up the One Hot encoding to analyze each neighborhood

In [86]:
pd.set_option('display.max_columns', None)
# one hot encoding
annapolis_onehot = pd.get_dummies(annapolis_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighbs = annapolis_venues['Neighborhood']
annapolis_onehot.insert(0,'Neighborhood',neighbs)
annapolis_onehot.head()

annapolis_onehot.head()

,Neighborhood,Airport,American Restaurant,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Convenience Store,Deli / Bodega,Dessert Shop,Fast Food Restaurant,Food,French Restaurant,Gas Station,Gastropub,Gift Shop,Gourmet Shop,Grocery Store,Harbor / Marina,History Museum,Hotel,Ice Cream Shop,Irish Pub,Japanese Restaurant,Liquor Store,Locksmith,Medical Center,Mexican Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Restaurant,Sandwich Place,Steakhouse,Supplement Shop,Sushi Restaurant,Taco Place,Wine Bar,Yoga Studio
0,"Annapolis Neck, Maryland",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Annapolis Neck, Maryland",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Annapolis Neck, Maryland",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Annapolis Neck, Maryland",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Annapolis Neck, Maryland",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


## This is to examine the new dataframe size

In [87]:
annapolis_onehot.shape

(112, 46)

## Next, I grouped the rows by neighborhood and took the mean of the frequency and occurence of each category

In [88]:
annapolis_grouped=annapolis_onehot.groupby('Neighborhood').mean().reset_index()
annapolis_grouped.head()

,Neighborhood,Airport,American Restaurant,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Convenience Store,Deli / Bodega,Dessert Shop,Fast Food Restaurant,Food,French Restaurant,Gas Station,Gastropub,Gift Shop,Gourmet Shop,Grocery Store,Harbor / Marina,History Museum,Hotel,Ice Cream Shop,Irish Pub,Japanese Restaurant,Liquor Store,Locksmith,Medical Center,Mexican Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Restaurant,Sandwich Place,Steakhouse,Supplement Shop,Sushi Restaurant,Taco Place,Wine Bar,Yoga Studio
0,"Annapolis Neck, Maryland",0.000000,0.066667,0.000000,0.000000,0.1,0.033333,0.000000,0.000000,0.066667,0.033333,0.000000,0.033333,0.033333,0.000000,0.033333,0.033333,0.000000,0.033333,0.033333,0.033333,0.000000,0.033333,0.066667,0.033333,0.033333,0.033333,0.033333,0.0,0.0,0.0,0.000000,0.0,0.033333,0.000000,0.000000,0.0,0.033333,0.0,0.033333,0.033333,0.000000,0.033333,0.033333,0.033333,0.0
1,"Arnold, Maryland",0.000000,0.066667,0.000000,0.000000,0.1,0.033333,0.000000,0.000000,0.066667,0.033333,0.000000,0.033333,0.033333,0.000000,0.033333,0.033333,0.000000,0.033333,0.033333,0.033333,0.000000,0.033333,0.066667,0.033333,0.033333,0.033333,0.033333,0.0,0.0,0.0,0.000000,0.0,0.033333,0.000000,0.000000,0.0,0.033333,0.0,0.033333,0.033333,0.000000,0.033333,0.033333,0.033333,0.0
2,"Cape St. Claire, Maryland",0.000000,0.066667,0.000000,0.000000,0.1,0.033333,0.000000,0.000000,0.066667,0.033333,0.000000,0.033333,0.033333,0.000000,0.033333,0.033333,0.000000,0.033333,0.033333,0.033333,0.000000,0.033333,0.066667,0.033333,0.033333,0.033333,0.033333,0.0,0.0,0.0,0.000000,0.0,0.033333,0.000000,0.000000,0.0,0.033333,0.0,0.033333,0.033333,0.000000,0.033333,0.033333,0.033333,0.0
3,"Edgewater, Maryland",0.066667,0.000000,0.066667,0.066667,0.0,0.000000,0.066667,0.066667,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.0,0.0,0.0,0.066667,0.0,0.000000,0.066667,0.066667,0.0,0.000000,0.0,0.066667,0.000000,0.066667,0.000000,0.000000,0.000000,0.0
4,"Hillsmere Shores, Maryland",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.5,0.000000,0.000000,0.000000,0.5,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


## This is confirming the new size of the data frame.

In [89]:
annapolis_grouped.shape

(7, 46)

## Next, I printed each neighborhood with the top 5 most common venues.

In [91]:
num_top_venues = 5

for hood in annapolis_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = annapolis_grouped[annapolis_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Annapolis Neck, Maryland----
                 venue  freq
0                  Bar  0.10
1       History Museum  0.07
2  American Restaurant  0.07
3          Coffee Shop  0.07
4                 Food  0.03


----Arnold, Maryland----
                 venue  freq
0                  Bar  0.10
1       History Museum  0.07
2  American Restaurant  0.07
3          Coffee Shop  0.07
4                 Food  0.03


----Cape St. Claire, Maryland----
                 venue  freq
0                  Bar  0.10
1       History Museum  0.07
2  American Restaurant  0.07
3          Coffee Shop  0.07
4                 Food  0.03


----Edgewater, Maryland----
                venue  freq
0             Airport  0.07
1   Convenience Store  0.07
2      Ice Cream Shop  0.07
3  Mexican Restaurant  0.07
4       Grocery Store  0.07


----Hillsmere Shores, Maryland----
        venue  freq
0        Park   0.5
1  Playground   0.5
2     Airport   0.0
3    Pharmacy   0.0
4   Irish Pub   0.0


----Parole, Maryland----


## Wrote a function to sort the venues in descending order

In [92]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Next, I created a new dataframe and displayed the top 10 venues for each neighborhood.

In [96]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = annapolis_grouped['Neighborhood']

for ind in np.arange(annapolis_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(annapolis_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Annapolis Neck, Maryland",Bar,History Museum,American Restaurant,Coffee Shop,Sushi Restaurant,Japanese Restaurant,French Restaurant,Dessert Shop,Gastropub,Gift Shop
1,"Arnold, Maryland",Bar,History Museum,American Restaurant,Coffee Shop,Sushi Restaurant,Japanese Restaurant,French Restaurant,Dessert Shop,Gastropub,Gift Shop
2,"Cape St. Claire, Maryland",Bar,History Museum,American Restaurant,Coffee Shop,Sushi Restaurant,Japanese Restaurant,French Restaurant,Dessert Shop,Gastropub,Gift Shop
3,"Edgewater, Maryland",Airport,Pizza Place,Convenience Store,Ice Cream Shop,Mexican Restaurant,Chinese Restaurant,Pharmacy,Breakfast Spot,Gas Station,Sandwich Place
4,"Hillsmere Shores, Maryland",Playground,Park,Yoga Studio,Convenience Store,Gift Shop,Gastropub,Gas Station,French Restaurant,Food,Fast Food Restaurant
5,"Parole, Maryland",Yoga Studio,Restaurant,Liquor Store,Medical Center,Deli / Bodega,Gift Shop,Gastropub,Gas Station,French Restaurant,Food
6,"St. Margaret's, Maryland",Locksmith,Yoga Studio,Deli / Bodega,Gourmet Shop,Gift Shop,Gastropub,Gas Station,French Restaurant,Food,Fast Food Restaurant


## Next, I ran k-means to cluster the neighborhood into 5 clusters.

In [101]:
kclusters=5
annapolis_grouped_clustering=annapolis_grouped.drop('Neighborhood',1)
kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(annapolis_grouped_clustering)
kmeans.labels_[0:5]

array([1, 1, 1, 4, 0])

## This is to create the new dataframe that includes the cluster in addition to the top 10 venues for each neighborhood.

In [102]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

annapolis_merged = annapolis

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
annapolis_merged = annapolis_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

annapolis_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Annapolis Neck, Maryland",38.980694,-76.488923,1,Bar,History Museum,American Restaurant,Coffee Shop,Sushi Restaurant,Japanese Restaurant,French Restaurant,Dessert Shop,Gastropub,Gift Shop
1,"Arnold, Maryland",38.980694,-76.488923,1,Bar,History Museum,American Restaurant,Coffee Shop,Sushi Restaurant,Japanese Restaurant,French Restaurant,Dessert Shop,Gastropub,Gift Shop
2,"Cape St. Claire, Maryland",38.980694,-76.488923,1,Bar,History Museum,American Restaurant,Coffee Shop,Sushi Restaurant,Japanese Restaurant,French Restaurant,Dessert Shop,Gastropub,Gift Shop
3,"Edgewater, Maryland",38.943558,-76.567270,4,Airport,Pizza Place,Convenience Store,Ice Cream Shop,Mexican Restaurant,Chinese Restaurant,Pharmacy,Breakfast Spot,Gas Station,Sandwich Place
4,"Hillsmere Shores, Maryland",38.939790,-76.495360,0,Playground,Park,Yoga Studio,Convenience Store,Gift Shop,Gastropub,Gas Station,French Restaurant,Food,Fast Food Restaurant


In [103]:
annapolis_merged=annapolis_merged.dropna(subset=['Cluster Labels'])

In [104]:
annapolis_merged[annapolis_merged['Cluster Labels'].isnull()]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [105]:
annapolis_merged=annapolis_merged.astype({'Cluster Labels':'int32'})
annapolis_merged.dtypes

Neighborhood               object
Latitude                  float64
Longitude                 float64
Cluster Labels              int32
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

## Next, I created a map utilizing Folium to visualize the resulting clusters

In [106]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(annapolis_merged['Latitude'], annapolis_merged['Longitude'], annapolis_merged['Neighborhood'], annapolis_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## The next 5 lines of code were to examine each cluster to determine the descriminating venue categories.

In [116]:
annapolis_merged.loc[annapolis_merged['Cluster Labels'] == 0, annapolis_merged.columns[[0,1,2,3,4] + list(range(5, annapolis_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"Hillsmere Shores, Maryland",38.93979,-76.49536,0,Playground,Park,Yoga Studio,Convenience Store,Gift Shop,Gastropub,Gas Station,French Restaurant,Food,Fast Food Restaurant


In [117]:
annapolis_merged.loc[annapolis_merged['Cluster Labels'] == 1, annapolis_merged.columns[[0,1,2,3,4] + list(range(5, annapolis_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Annapolis Neck, Maryland",38.980694,-76.488923,1,Bar,History Museum,American Restaurant,Coffee Shop,Sushi Restaurant,Japanese Restaurant,French Restaurant,Dessert Shop,Gastropub,Gift Shop
1,"Arnold, Maryland",38.980694,-76.488923,1,Bar,History Museum,American Restaurant,Coffee Shop,Sushi Restaurant,Japanese Restaurant,French Restaurant,Dessert Shop,Gastropub,Gift Shop
2,"Cape St. Claire, Maryland",38.980694,-76.488923,1,Bar,History Museum,American Restaurant,Coffee Shop,Sushi Restaurant,Japanese Restaurant,French Restaurant,Dessert Shop,Gastropub,Gift Shop


In [118]:
annapolis_merged.loc[annapolis_merged['Cluster Labels'] == 2, annapolis_merged.columns[[0,1,2,3,4] + list(range(5, annapolis_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"St. Margaret's, Maryland",39.018739,-76.466609,2,Locksmith,Yoga Studio,Deli / Bodega,Gourmet Shop,Gift Shop,Gastropub,Gas Station,French Restaurant,Food,Fast Food Restaurant


In [119]:
annapolis_merged.loc[annapolis_merged['Cluster Labels'] == 3, annapolis_merged.columns[[0,1,2,3,4] + list(range(5, annapolis_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,"Parole, Maryland",38.997037,-76.511214,3,Yoga Studio,Restaurant,Liquor Store,Medical Center,Deli / Bodega,Gift Shop,Gastropub,Gas Station,French Restaurant,Food


In [120]:
annapolis_merged.loc[annapolis_merged['Cluster Labels'] == 4, annapolis_merged.columns[[0,1,2,3,4] + list(range(5, annapolis_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"Edgewater, Maryland",38.943558,-76.56727,4,Airport,Pizza Place,Convenience Store,Ice Cream Shop,Mexican Restaurant,Chinese Restaurant,Pharmacy,Breakfast Spot,Gas Station,Sandwich Place
